In [1]:
import pandas as pd

KEYWORD_FILE = "keywords/keywords-prototype-21-04-22.xlsx"
DATA_VERSION = "21-04-22"

df_keywords = pd.read_excel(KEYWORD_FILE)

df_keywords = df_keywords.loc[~df_keywords["KeywordID"].isna()]
df_keywords["KeywordID"] = df_keywords["KeywordID"].astype(int)


/root/.local/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
languages = set(df_keywords.columns)
languages.remove('FlagWithoutGermany')
languages.remove('KeywordID')
languages


{'BG',
 'CS',
 'DA',
 'DE',
 'EL',
 'EN',
 'ES',
 'ET',
 'FI',
 'FR',
 'HR',
 'HU',
 'IT',
 'LI',
 'LV',
 'NL',
 'PL',
 'PT',
 'RO',
 'SK',
 'SL',
 'SV'}

In [4]:
from db_stuff import db_connector

db = db_connector()
psql_insert_copy = db.psql_insert_copy


In [5]:
version_id = db.get_version_id(DATA_VERSION)[0]

for language in languages:
    if 'keywords' in globals():
        del keywords
    print(language)
    language_id = db.get_language_id(language)[0]
    keywords = df_keywords[['KeywordID', language, 'FlagWithoutGermany']].copy()
    keywords.loc[:, ('FlagWithoutGermany')] = keywords.apply(
        lambda row: 0 if row.isna()["FlagWithoutGermany"]
        else 1,
        axis=1
    ).astype('bool')
    keywords.loc[:,'language_id'] = language_id
    keywords.loc[:,'version_id'] = version_id
    keywords.rename(columns = {'KeywordID':'keyword_id', 'FlagWithoutGermany':'without_germany', language:'keyword', 'index':'id'}, inplace=True)
    keywords = keywords[['keyword_id', 'language_id', 'version_id', 'without_germany', 'keyword']]
    keywords.to_sql('l_keyword', db.engine, schema='trend', method=psql_insert_copy, if_exists='append', index=False)



ES
PL
LI
SV
IT
PT
ET
RO
SK
EL
FR
FI
DE
HR
LV
DA
CS
BG
SL
EN
HU
NL
